<a href="https://colab.research.google.com/github/jeffreyfeng99/SYDE_522_A3/blob/master/SYDE_522_Assignment_3_testingincludedwithpath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Submission Notes
Group members:

Jeffrey Feng, 20704800

Joey Kuang, 20726074

# Import statements

In [1]:
import os
import pandas as pd
import numpy as np
import random
from PIL import Image
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Function
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torchvision import datasets
from torchvision import models
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Config

In [3]:
cuda = True
cudnn.benchmark = True
LR = 1e-3
BATCH_SIZE = 64
IMAGE_SIZE = 224 #227
FT_OUT_SIZE = 512
N_EPOCH = 10 #100

# Data loader

In [4]:
dataset_root = '/content/drive/MyDrive/4B/SYDE-522/data'
output_root = '/content/drive/MyDrive/4B/SYDE-522/submission'
source_dataset_name = 'train_set'
target_dataset_name = 'test_set'

source_image_root = os.path.join(dataset_root, source_dataset_name)
target_image_root = os.path.join(dataset_root, target_dataset_name)

train_label_list = os.path.join(dataset_root, 'train_labels.csv')

In [5]:
class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list=None, transform=None):
        self.root = data_root
        self.transform = transform

        # we only pass data_list if it's training set
        if data_list is not None:
            df = pd.read_csv(data_list)
            self.img_paths = df['dir'].to_list()

            if 'label2' in df.columns:
                self.img_labels = df['label2'].to_list()
            else: 
                self.img_labels = ['0' for i in range(len(self.img_paths))]
        else:
            # Walk through test folder - we don't need labels
            self.img_paths = [f for root,dirs,files in os.walk(data_root) for f in files if f.endswith('.png')]
            self.img_labels = ['0' for i in range(len(self.img_paths))]

        self.n_data = len(self.img_paths)

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item%self.n_data], self.img_labels[item%self.n_data]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels, img_paths

    def __len__(self):
        return self.n_data

In [6]:
# Preprocess data
def preprocess_fn(mu=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)):
  img_transform = transforms.Compose([
      transforms.Resize(IMAGE_SIZE),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) # why isn't this (0.1307,) and (0.3081,) like in test?
  ])

  return img_transform

def prep_dataloader(image_root, label_list=None, img_transform=None, 
                    drop_last=False, shuffle=True):
    dataset = GetLoader(
        data_root=image_root,
        data_list=label_list,
        transform=img_transform
    )

    dataloader = data.DataLoader(
        dataset=dataset,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        num_workers=8,
        drop_last=drop_last)
    
    return dataset, dataloader


# Model definition

In [7]:
# if False, then we are feature extracting
def set_parameter_requires_grad(model, finetune):
    for param in model.parameters():
        param.requires_grad = finetune

In [8]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None


class CNNModel(nn.Module):

    def __init__(self, model_name="resnet18"):
        super(CNNModel, self).__init__()

        if model_name == "resnet18":
            self.feature = models.resnet18(pretrained=True) 
            self.feature.fc = nn.Identity()
        elif model_name == "vgg11":
            self.feature = models.vgg11_bn(pretrained=True) 
            self.feature.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1)) # original is (7,7)
            self.feature.classifier = nn.Identity()
        else:
            # Need some default model?
            self.feature = nn.Sequential()
            self.feature.add_module('f_conv1', nn.Conv2d(3, 64, kernel_size=5))
            self.feature.add_module('f_bn1', nn.BatchNorm2d(64))
            self.feature.add_module('f_pool1', nn.MaxPool2d(2))
            self.feature.add_module('f_relu1', nn.ReLU(True))
            self.feature.add_module('f_conv2', nn.Conv2d(64, 128, kernel_size=3))
            self.feature.add_module('f_bn2', nn.BatchNorm2d(128))
            self.feature.add_module('f_drop1', nn.Dropout2d())
            self.feature.add_module('f_pool2', nn.MaxPool2d(2))
            self.feature.add_module('f_relu2', nn.ReLU(True))
            self.feature.add_module('f_conv3', nn.Conv2d(128, 256, kernel_size=3))
            self.feature.add_module('f_bn3', nn.BatchNorm2d(256))
            self.feature.add_module('f_drop3', nn.Dropout2d())
            self.feature.add_module('f_pool3', nn.MaxPool2d(2))
            self.feature.add_module('f_relu4', nn.ReLU(True))
            self.feature.add_module('f_conv4', nn.Conv2d(256, 256, kernel_size=3))
            self.feature.add_module('f_bn4', nn.BatchNorm2d(256))
            self.feature.add_module('f_drop4', nn.Dropout2d())
            self.feature.add_module('f_pool4', nn.MaxPool2d(2))
            self.feature.add_module('f_relu4', nn.ReLU(True))
            self.feature.add_module('f_conv5', nn.Conv2d(256, 512, kernel_size=3))
            self.feature.add_module('f_bn5', nn.BatchNorm2d(512))
            self.feature.add_module('f_drop5', nn.Dropout2d())
            self.feature.add_module('f_pool5', nn.MaxPool2d(2))
            self.feature.add_module('f_relu5', nn.ReLU(True))

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(FT_OUT_SIZE, 100))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout2d())
        self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(100, 7))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax(dim=1))

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(FT_OUT_SIZE, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 4))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, IMAGE_SIZE, IMAGE_SIZE)
        feature = self.feature(input_data)
        feature = feature.view(-1, FT_OUT_SIZE)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

# Test pipeline

In [9]:
def test(net, epoch):

    # load data
    img_transform_source = preprocess_fn(mu=(0.1307,), std=(0.3081,))
    img_transform_target = preprocess_fn()

    dataset_source, dataloader_source = prep_dataloader(
        image_root=os.path.join(source_image_root, 'train_set'),
        label_list=train_label_list,
        img_transform=img_transform_source,
        shuffle=False
    )

    dataset_target, dataloader_target = prep_dataloader(
        image_root=os.path.join(target_image_root, 'test_set'),
        img_transform=img_transform_target,
        shuffle=False
    )

    net.eval()

    if cuda:
        net = net.cuda()

    train_pths, train_preds = inference(net, dataloader_source, cuda=cuda, alpha=alpha)
    train_results = pd.DataFrame({'id': train_pths, 'label': train_preds})
    train_results_pth = os.path.join(output_root, '%s_train_epoch%s.csv' % (datetime.now().strftime("%m%d%Y"), epoch))
    train_results.to_csv(train_results_pth, index=False)

    test_pths, test_preds = inference(net, dataloader_target, cuda=cuda, alpha=alpha)
    test_results = pd.DataFrame({'id': test_pths, 'label': test_preds})
    test_results_pth = os.path.join(output_root, '%s_test_epoch%s.csv' % (datetime.now().strftime("%m%d%Y"), epoch))
    test_results.to_csv(test_results_pth, index=False)

    print('epoch: %d, accuracy of the train dataset: %f' % (epoch, compare(train_label_list, train_results_pth)))

    # Secret
    test_label_list = os.path.join(dataset_root, 'dummy_test_labels.csv')
    print('epoch: %d, accuracy of the test dataset: %f' % (epoch, compare(test_label_list, test_results_pth)))

def inference(net, dataloader, cuda=True, alpha=0.0):
    preds = []
    pths = []
    for input_img, _, img_paths in dataloader: 

        if cuda:
            input_img = input_img.cuda()

        class_output, _ = net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        pths = pths + list(img_paths)
        preds = preds + list(pred.squeeze(1).cpu().numpy())
    return pths, preds

def compare(true_labels, predicted_labels):
    combined_df = pd.read_csv(true_labels)
    predicted_df = pd.read_csv(predicted_labels)

    combined_df['label'] = combined_df['dir'].map(predicted_df.set_index('id')['label'])

    true_labels = np.array(combined_df['label2'].to_list())
    pred_labels = np.array(combined_df['label'].to_list())

    return np.sum(true_labels == pred_labels) / len(true_labels)

# Training pipeline

In [10]:
manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# load data
img_transform_source = preprocess_fn()
img_transform_target = preprocess_fn()

dataset_source, dataloader_source = prep_dataloader(
    image_root=os.path.join(source_image_root, 'train_set'), # TODO should we unnest
    label_list=train_label_list,
    img_transform=img_transform_source
)

dataset_target, dataloader_target = prep_dataloader(
    image_root=os.path.join(target_image_root, 'test_set'),
    img_transform=img_transform_target
)

# load model
my_net = CNNModel()

# setup optimizer
optimizer = optim.Adam(my_net.parameters(), lr=LR)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

set_parameter_requires_grad(my_net, True)

# training
for epoch in tqdm(range(N_EPOCH)):

    len_dataloader = max(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / N_EPOCH / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = data_source_iter.next()
        s_img, s_label, _ = data_source

        my_net.zero_grad()
        batch_size = len(s_label)

        input_img = torch.FloatTensor(batch_size, 3, IMAGE_SIZE, IMAGE_SIZE)
        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)

        class_output, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        if i == len(dataloader_target):
            data_target_iter = iter(dataloader_target)
        data_target = data_target_iter.next()
        t_img, _, _ = data_target

        batch_size = len(t_img) # TODO: why?

        input_img = torch.FloatTensor(batch_size, 3, IMAGE_SIZE, IMAGE_SIZE)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)

        _, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        i += 1

        print('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
            % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
                err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

    # torch.save(my_net, 'output/model_epoch_{}.pth'.format(epoch))
    test(my_net, epoch)
    my_net.train()

print('done')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0, [iter: 1 / all 95], err_s_label: 1.967160, err_s_domain: 1.366492, err_t_domain: 1.503473
epoch: 0, [iter: 2 / all 95], err_s_label: 1.854151, err_s_domain: 1.338172, err_t_domain: 1.445730
epoch: 0, [iter: 3 / all 95], err_s_label: 1.558821, err_s_domain: 1.265486, err_t_domain: 1.392847
epoch: 0, [iter: 4 / all 95], err_s_label: 1.517837, err_s_domain: 1.231521, err_t_domain: 1.351936
epoch: 0, [iter: 5 / all 95], err_s_label: 1.543024, err_s_domain: 1.167236, err_t_domain: 1.295579
epoch: 0, [iter: 6 / all 95], err_s_label: 1.492177, err_s_domain: 1.148198, err_t_domain: 1.277937
epoch: 0, [iter: 7 / all 95], err_s_label: 1.391278, err_s_domain: 1.084720, err_t_domain: 1.231409
epoch: 0, [iter: 8 / all 95], err_s_label: 1.338994, err_s_domain: 1.085517, err_t_domain: 1.217844
epoch: 0, [iter: 9 / all 95], err_s_label: 1.275766, err_s_domain: 1.026999, err_t_domain: 1.188297
epoch: 0, [iter: 10 / all 95], err_s_label: 1.273778, err_s_domain: 1.005067, err_t_domain: 1.148619

 10%|█         | 1/10 [07:36<1:08:31, 456.78s/it]

...

epoch: 7, accuracy of the train dataset: 0.915045
epoch: 7, accuracy of the test dataset: 0.520998
epoch: 8, [iter: 1 / all 95], err_s_label: 0.172065, err_s_domain: 0.729607, err_t_domain: 0.706563
epoch: 8, [iter: 2 / all 95], err_s_label: 0.167667, err_s_domain: 0.708972, err_t_domain: 0.706918
epoch: 8, [iter: 3 / all 95], err_s_label: 0.197238, err_s_domain: 0.729916, err_t_domain: 0.721756
epoch: 8, [iter: 4 / all 95], err_s_label: 0.050681, err_s_domain: 0.701596, err_t_domain: 0.719931
epoch: 8, [iter: 5 / all 95], err_s_label: 0.088658, err_s_domain: 0.698992, err_t_domain: 0.707040
epoch: 8, [iter: 6 / all 95], err_s_label: 0.115543, err_s_domain: 0.690399, err_t_domain: 0.730394
epoch: 8, [iter: 7 / all 95], err_s_label: 0.139526, err_s_domain: 0.686203, err_t_domain: 0.740921
epoch: 8, [iter: 8 / all 95], err_s_label: 0.118266, err_s_domain: 0.690996, err_t_domain: 0.733461
epoch: 8, [iter: 9 / all 95], err_s_label: 0.041140, err_s_domain: 0.702796, err_t_domain: 0.719108
e

 90%|█████████ | 9/10 [25:12<02:17, 137.59s/it]

epoch: 8, accuracy of the train dataset: 0.866381
epoch: 8, accuracy of the test dataset: 0.463731
epoch: 9, [iter: 1 / all 95], err_s_label: 0.119331, err_s_domain: 0.696470, err_t_domain: 0.680370
epoch: 9, [iter: 2 / all 95], err_s_label: 0.133579, err_s_domain: 0.699807, err_t_domain: 0.687464
epoch: 9, [iter: 3 / all 95], err_s_label: 0.085530, err_s_domain: 0.697391, err_t_domain: 0.692285
epoch: 9, [iter: 4 / all 95], err_s_label: 0.167398, err_s_domain: 0.695729, err_t_domain: 0.667569
epoch: 9, [iter: 5 / all 95], err_s_label: 0.132881, err_s_domain: 0.698852, err_t_domain: 0.689089
epoch: 9, [iter: 6 / all 95], err_s_label: 0.060202, err_s_domain: 0.697743, err_t_domain: 0.688245
epoch: 9, [iter: 7 / all 95], err_s_label: 0.183778, err_s_domain: 0.704539, err_t_domain: 0.683368
epoch: 9, [iter: 8 / all 95], err_s_label: 0.119477, err_s_domain: 0.693865, err_t_domain: 0.681751
epoch: 9, [iter: 9 / all 95], err_s_label: 0.122966, err_s_domain: 0.699729, err_t_domain: 0.681255
e

100%|██████████| 10/10 [27:24<00:00, 164.47s/it]

epoch: 9, accuracy of the train dataset: 0.781260
epoch: 9, accuracy of the test dataset: 0.536269
done
